In [2]:
%%time
# sized_snp.py
"""Program to down-size option chains
Date: 24-June-2019
Rev: 1.0
Time taken to execute fully: 13 mins"""

from z_helper import *
util.startLoop()

from chains_snp import *
from ohlcs import *


# # from json
a = assign_var('common') + assign_var('snp')
for v in a:
    exec(v)

#... check for chains and ohlcs
if path.isfile(fspath+'chains_snp.pkl'):
    df_chains = pd.read_pickle(fspath+'snp_chains.pkl')
else:
    with get_connected('snp', 'live') as ib:
        df_chains = get_chains(ib)

if path.isfile(fspath+'ohlcs.pkl'):
    df_ohlcs = pd.read_pickle(fspath+'ohlcs.pkl')

else:
    id_sym = df_chains.set_index('undId').symbol.to_dict()

    with get_connected('snp', 'live') as ib:
        df_ohlcs = ohlcs(ib, id_sym, fspath, logpath)

    df_ohlcs.to_pickle(fspath+'ohlcs.pkl')

# # ib = get_connected('snp', 'live')
# df_chains = pd.read_pickle(fspath+'chains_snp.pkl')
# df_ohlcs = pd.read_pickle(fspath+'ohlcs.pkl')

# log to size_chains.log
with open(logpath+'size_chains.log', 'w'):
    pass # clear the run log
util.logToFile(logpath+'size_chains.log')

#... Remove chains not meeting put and call std filter

# get dte and remove those greater than maxdte
df_chains = df_chains.assign(dte=df_chains.expiry.apply(get_dte))                    
df_chains = df_chains[df_chains.dte <= maxdte]

# generate std dataframe
df = df_ohlcs[['symbol', 'stDev']]  # lookup dataframe
df = df.assign(dte=df.groupby('symbol').cumcount()) # get the cumulative count for location as dte
df.set_index(['symbol', 'dte'])

df1 = df_chains[['symbol', 'dte']]  # data to be looked at
df2 = df1.drop_duplicates()  # remove duplicates

df_std = df2.set_index(['symbol', 'dte']).join(df.set_index(['symbol', 'dte']))

# join to get std in chains
df_chainstd = df_chains.set_index(['symbol', 'dte']).join(df_std).reset_index()

und_contracts = [Stock(symbol, exchange=exchange, currency='USD') for symbol in df_chainstd.symbol.unique()]
with get_connected('snp', 'live') as ib:
    und_quals = ib.qualifyContracts(*und_contracts)
    tickers = ib.reqTickers(*und_quals)

uprice_dict = {u.contract.conId: u.marketPrice() for u in tickers}

df_chainstd = df_chainstd.assign(undPrice=df_chainstd.undId.astype('int').map(uprice_dict))

# make puts and calls dataframe with std filter
df_puts = df_chainstd[df_chainstd.strike < (df_chainstd.undPrice-(df_chainstd.stDev*putstdmult))]
df_puts = df_puts.assign(right = 'P')

df_calls = df_chainstd[df_chainstd.strike > (df_chainstd.undPrice+(df_chainstd.stDev*callstdmult))]
df_calls = df_calls.assign(right = 'C')

df_opt = pd.concat([df_puts, df_calls], sort=False).reset_index(drop=True)

# get lo52 and hi52
df_opt = df_opt.set_index('symbol').join(df_ohlcs.groupby('symbol')
                                         .close.agg(['min', 'max'])
                                         .rename(columns={'min': 'lo52', 'max': 'hi52'})).reset_index()

# make (df and dte) tuple for fallrise
tup4fr = [(df_ohlcs[df_ohlcs.symbol == s.symbol], s.dte) 
          for s in df_opt[['symbol', 'dte']].drop_duplicates().itertuples()]

# get the fallrise and put it into a dataframe
fr = [fallrise(*t) for t in tup4fr]
df_fr = pd.DataFrame(fr, columns=['symbol', 'dte', 'fall', 'rise' ])

# merge with df_opt
df_opt = pd.merge(df_opt, df_fr, on=['symbol', 'dte'])

# make reference strikes from fall_rise
df_opt = df_opt.assign(strikeRef = np.where(df_opt.right == 'P', 
                                            df_opt.undPrice-df_opt.fall, 
                                            df_opt.undPrice+df_opt.rise))

# get the strikes closest to the reference strikes
df_opt = df_opt.groupby(['symbol', 'dte']) \
                         .apply(lambda g: g.iloc[abs(g.strike-g.strikeRef) \
                                                 .argsort()[:nBand]])

df_opt = df_opt.set_index('symbol').reset_index()

# get the option contracts

opt_list = [Option(i.symbol, i.expiry, i.strike, i.right, 'SMART') for i in df_opt[['symbol', 'expiry', 'strike', 'right']].itertuples()]

util.logToFile(logpath+'test.log') # prevents unknown contract errors in console

opt_contracts = []
with get_connected('snp', 'live') as ib:
    print("Qualifying option contracts ...(2 to 8 mins)")
    opt_contracts = ib.qualifyContracts(*opt_list)

# integrate optId with df_opt and remove df_opt without optId
dfq = util.df(opt_contracts).iloc[:, 1:6]
dfq.columns=['optId', 'symbol', 'expiry', 'strike', 'right'] # rename columns
df_opt=df_opt.merge(dfq, on=['symbol', 'expiry', 'strike', 'right'], how='left')
df_opt = df_opt[~df_opt.optId.isnull()]
df_opt = df_opt.assign(optId=df_opt.optId.astype('int'))

# get the option prices
with get_connected('snp', 'live') as ib:
    ticker = ib.reqTickers(*opt_contracts)

df_prices = pd.DataFrame({t.contract.conId: {'bid':t.bid, 'ask':t.ask, 'close':t.close} for t in ticker}).T

# ...get margins

# prepare the lots
idlot_idx = df_opt[['optId', 'lot']].set_index('optId').to_dict('index')
idlot = {k: Order(action='SELL', orderType='MKT', totalQuantity=v['lot']/100, whatIf=True) for k, v in idlot_idx.items()}

co = [(c, idlot[c.conId]) for c in opt_contracts]

# co = co[:110]  # DATA LIMITER !!!
coblks = [co[i: i+blk] for i in range(0, len(co), blk)]

m = {} # empty dictionary to collect outputs of getMarginAsync
with get_connected('snp', 'live') as ib:
    async def coro(coblk):
        with tqdm(total=len(coblk), file=sys.stdout, unit=' symexpiry') as tqm:
            for c, o in coblk:
                tqm.set_description(f"IBKR margins for  {c.localSymbol.ljust(22)}")
                m.update(await getMarginAsync(ib, c, o))
                tqm.update(1)
            return m

# run co-routines to get the margins        
for coblk in coblks:
    with get_connected('snp', 'live') as ib:
        asyncio.run(coro(coblk))

# put margins to df_opt
m_dict = {i: float(j.initMarginChange) for i, j in {k: v for k, v in m.items() if v}.items() if str(j) != 'nan'}

df_margin = pd.DataFrame.from_dict(m_dict, orient='index', columns=['margin'])

df_opt = df_opt.set_index('optId').join(df_prices).join(df_margin).reset_index()

df_opt = grp_opts(df_opt.assign(rom=df_opt.close/df_opt.margin*365/df_opt.dte*df_opt.lot))

df_opt.to_pickle(fspath+'sized.pkl')

grp_opts(df_opt).to_excel(fspath+'sized_snp.xlsx', index=False, freeze_panes=(1,2))

Getting strikes & expiries for JNJ      : 100%|█████████████████████████████████| 91/91 [00:07<00:00, 11.91 contract/s]
Qualifying option contracts ...(2 to 8 mins)
IBKR margins for  XOM   190816P00055000 : 100%|████████████████████████████████| 16/16 [00:03<00:00,  4.54 symexpiry/s]
Wall time: 7min 59s


In [ ]:
# test code for snp sized

from z_helper import *
util.startLoop()

# from json (for logpath)
a = assign_var('snp')
for v in a:
    exec(v)

from chains_snp import *
from ohlcs import *

if __name__ == '__main__':
    
    #... check for snp_chains.pkl
    if path.isfile(fspath+'snp_chains.pkl'):
        df_chains = pd.read_pickle(fspath+'snp_chains.pkl')
    else:
        with get_connected('snp', 'live') as ib:
            df_chains = get_chains(ib)

#     df_chains = df_chains[df_chains.symbol.isin(['BANKNIFTY', 'PNB'])]  # DATA LIMITER!!!

    if path.isfile(fspath+'ohlcs.pkl'):
        df_ohlcs = pd.read_pickle(fspath+'ohlcs.pkl')
        
    else:
        id_sym = df_chains.set_index('undId').symbol.to_dict()
    
    with get_connected('snp', 'live') as ib:
        df_ohlcs = ohlcs(ib, id_sym, fspath, logpath)

    df_ohlcs.to_pickle(fspath+'ohlcs.pkl')
    
    market = 'snp'
    